In [3]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

# EU KLEMS

Database description: 
    The EU KLEMS project aimed at creating a database on measures of economic growth, productivity, employment creation, capital formation and technological change at the industry level for all European Union member states from 1970 onwards. The database will provide an important input to policy evaluation, in particular for the assessment of the goals concerning competitiveness and economic growth potential as established by the Lisbon and Barcelona summit goals.
    
The input measures include various categories of capital, labour, energy, material and service inputs. Productivity measures have also been developed, in particular with growth accounting techniques. Several measures on knowledge creation have also been constructed. 
   
Purpose: The EU KLEMS Database can be used for analytical and policy-related purposes, in particular by studying the relationship between skill formation, technological progress and innovation on the one hand, and productivity, on the other. To facilitate this type of analysis a link will has been sought with existing micro (firm level) databases. The balance in academic, statistical and policy input in this project was realised by the participation of 15 organisations from across the EU, representing a mix of academic institutions and national economic policy research institutes and with the support from various statistical offices and the OECD.

The code downloads the EU KLEMS database's output and capital input files from www.euklems.net and convert it into a pandas dataframe.

In [4]:
HEADERS = \
    {'Host': 'www.euklems.net',
     'Connection': 'keep-alive',
     'Cache-Control': 'max-age=0',
     'Upgrade-Insecure-Requests': '1',
     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
     'Accept-Encoding': 'gzip, deflate',
     'Accept-Language': 'hu-HU,hu;q=0.9,en-US;q=0.8,en;q=0.7',
     'If-None-Match': "3ec-5724883dc06c0",
    'If-Modified-Since': 'Tue, 31 Jul 2018 09:50:59 GMT'}
homelink = 'http://www.euklems.net/index_TCB_201807.shtml'
home = requests.get(homelink,headers = HEADERS)

In [5]:
excel_files = [i.get('href') for i in BeautifulSoup(home.content).find_all('a')\
               if i.get('href') is not None and '.xlsx' in i.get('href')\
               and 'labour' not in i.get('href')]

Csinál egy key táblát, és exportálja .csv-ben, ez már megvan a mappámban csak be kell hívni

keys_1 = pd.read_excel('http://www.euklems.net/' + excel_files[0])\
            .rename({'Austria':'Key','Unnamed: 1':'Variable'}, axis = 1)\
            .dropna(axis = 0, subset = ['Variable'])\
            .dropna(axis = 1, how = 'all')

keys_2 = pd.read_excel('http://www.euklems.net/' + excel_files[1])\
           .rename({'Austria':'Key','Unnamed: 1':'Variable'},axis = 1)

for j in range(5):
    for i in range(11):
        keys_2['Variable'][6 + j * 13 + i] =\
        keys_2['Key'][5 + j * 13] + ' of ' + keys_2['Variable'][6 + j * 13 + i]

keys_2 = keys_2.dropna(axis = 0, subset = ['Variable'])\
             .dropna(axis = 1, how = 'all')

pd.concat([keys_1, keys_2], sort = False)\
.to_csv('key_table.csv', index = False,header = True)

In [6]:
euklems_key = pd.read_csv('key_table.csv')

In [7]:
df = pd.DataFrame(columns = ['Country','Key','Year','Industry','Value'])
for j in tqdm(range(len(excel_files))):
    sheet_names = pd.ExcelFile('http://www.euklems.net/' + excel_files[j]).sheet_names
    sheet_names.remove('Notes')
    try:
        sheet_names.remove('Asset Breakdown')
    except:
        pass  
    excel_od = pd.read_excel('http://www.euklems.net/' + excel_files[j],sheet_name = sheet_names)
    df_con = pd.DataFrame(columns = ['Country','Key','Year','Industry','Value'])
    for _, e in enumerate(sheet_names):
        excel_od[e] = excel_od[e].dropna(axis = 1,how = 'all').dropna(axis = 0,how = 'all')\
                                 .drop('code', axis = 1)
        df_var = pd.melt(excel_od[e], id_vars = ["desc"])\
                 .rename({'variable':'Key','desc':'Industry','value':'Value'},axis = 1)
        if e == 'Deprate':
            df_var['Year'] = [None for i in range(df_var.shape[0])]
        else:
            df_var['Year'] = [df_var.at[i,'Key'][-4:] for i in range(df_var.shape[0])]
            df_var['Key'] = [e for i in range(df_var.shape[0])]
        df_con = pd.concat([df_con,df_var],sort = False)
    df_con['Country'] = list(pd.read_excel('http://www.euklems.net/' + excel_files[j]))[0]
    df = pd.concat([df,df_con],sort = False)

In [8]:
df_euklems = df.merge(euklems_key, how = 'left', on = 'Key')
df_euklems['Database'] = 'EU KLEMS'

In [19]:
df_euklems

,Country,Key,Year,Industry,Value,Variable,Database
0,Austria,VA,1995,TOTAL INDUSTRIES,158369,Gross value added at current basic prices (in ...,EU KLEMS
1,Austria,VA,1995,MARKET ECONOMY,117905,Gross value added at current basic prices (in ...,EU KLEMS
2,Austria,VA,1995,"AGRICULTURE, FORESTRY AND FISHING",3814.1,Gross value added at current basic prices (in ...,EU KLEMS
3,Austria,VA,1995,MINING AND QUARRYING,543.5,Gross value added at current basic prices (in ...,EU KLEMS
4,Austria,VA,1995,TOTAL MANUFACTURING,31602.4,Gross value added at current basic prices (in ...,EU KLEMS
5,Austria,VA,1995,"Food products, beverages and tobacco",3921.6,Gross value added at current basic prices (in ...,EU KLEMS
6,Austria,VA,1995,"Textiles, wearing apparel, leather and relate...",1669.5,Gross value added at current basic prices (in ...,EU KLEMS
7,Austria,VA,1995,Wood and paper products; printing and reprodu...,4167,Gross value added at current basic prices (in ...,EU KLEMS
8,Austria,VA,1995,Coke and refined petroleum products,673.5,Gross value added at current basic prices (in ...,EU KLEMS
9,Austria,VA,1995,Chemicals and chemical products,1964.8,Gross value added at current basic prices (in ...,EU KLEMS


In [25]:
df_euklems.groupby(['Country', 'Industry'])

Database  \
Country       Industry                                                       
Austria        ACCOMMODATION AND FOOD SERVICE ACTIVITIES          EU KLEMS   
               AGRICULTURE, FORESTRY AND FISHING                  EU KLEMS   
               ARTS, ENTERTAINMENT, RECREATION AND OTHER SERV...  EU KLEMS   
               Activities of extraterritorial organizations a...  EU KLEMS   
               Activities of households as employers; undiffe...  EU KLEMS   
               Arts, entertainment and recreation                 EU KLEMS   
               Basic metals and fabricated metal products, ex...  EU KLEMS   
               COMMUNITY SOCIAL AND PERSONAL SERVICES             EU KLEMS   
               CONSTRUCTION                                       EU KLEMS   
               Chemicals and chemical products                    EU KLEMS   
               Coke and refined petroleum products                EU KLEMS   
               ELECTRICITY, GAS AND WATER SUPPLY                  EU KLEMS   
               Education                                          EU KLEMS   
               Electrical and optical equipment                   EU KLEMS   
               FINANCIAL AND INSURANCE ACTIVITIES                 EU KLEMS   
               Food products, beverages and tobacco               EU KLEMS   
               Health and social work                             EU KLEMS   
               INFORMATION AND COMMUNICATION                      EU KLEMS   
               IT and other information services                  EU KLEMS   
               MARKET ECONOMY                                     EU KLEMS   
               MINING AND QUARRYING                               EU KLEMS   
               Machinery and equipment n.e.c.                     EU KLEMS   
               Other manufacturing; repair and installation o...  EU KLEMS   
               Other service activities                           EU KLEMS   
               PROFESSIONAL, SCIENTIFIC, TECHNICAL, ADMINISTR...  EU KLEMS   
               Postal and courier activities                      EU KLEMS   
               Public administration and defence; compulsory ...  EU KLEMS   
               Publishing, audiovisual and broadcasting activ...  EU KLEMS   
               REAL ESTATE ACTIVITIES                             EU KLEMS   
               Retail trade, except of motor vehicles and mot...  EU KLEMS   
...                                                                    ...   
United States Education                                           EU KLEMS   
              Electrical and optical equipment                    EU KLEMS   
              FINANCIAL AND INSURANCE ACTIVITIES                  EU KLEMS   
              Food products, beverages and tobacco                EU KLEMS   
              Health and social work                              EU KLEMS   
              INFORMATION AND COMMUNICATION                       EU KLEMS   
              IT and other information services                   EU KLEMS   
              MARKET ECONOMY                                      EU KLEMS   
              MINING AND QUARRYING                                EU KLEMS   
              Machinery and equipment n.e.c.                      EU KLEMS   
              Other manufacturing; repair and installation of...  EU KLEMS   
              Other service activities                            EU KLEMS   
              PROFESSIONAL, SCIENTIFIC, TECHNICAL, ADMINISTRA...  EU KLEMS   
              Postal and courier activities                       EU KLEMS   
              Public administration and defence; compulsory s...  EU KLEMS   
              Publishing, audiovisual and broadcasting activi...  EU KLEMS   
              REAL ESTATE ACTIVITIES                              EU KLEMS   
              Retail trade, except of motor vehicles and moto...  EU KLEMS   
              Rubber and plastics products, and other non-met...  EU KLEMS   
          

# Maddison Project Database 2018

Database description: The Maddison Project Database provides information on comparative economic growth and income levels over the very long run. The 2018 version of this database covers 169 countries and the period up to 2016.

The code downloads the Maddison Project 2018 database from https://www.rug.nl/ggdc/historicaldevelopment/maddison/releases/maddison-project-database-2018 and convert it into a pandas dataframe.

In [9]:
MP_sn = pd.ExcelFile('https://www.rug.nl/ggdc/historicaldevelopment/maddison/data/mpd2018.xlsx').sheet_names
MP = pd.read_excel('https://www.rug.nl/ggdc/historicaldevelopment/maddison/data/mpd2018.xlsx', sheet_name = MP_sn)

In [10]:
MP_key = MP[MP_sn[0]].dropna(axis = 0, subset = ['Unnamed: 1'])\
                     .rename({'Maddison Project Database (MPD) 2018':'Key','Unnamed: 1':'Variable'}, axis = 1)

In [11]:
df_mp = pd.DataFrame(columns = ['Country','Key','Year','Industry','Value'])
for i in range(2,5):
    df_var = pd.melt(MP[MP_sn[i]],id_vars = (MP_sn[i]))\
               .rename({MP_sn[i]:'Year','variable':'Country','value':'Value'}, axis = 1)
    df_var['Key'] = MP_sn[i]
    df_mp = pd.concat([df_mp,df_var],sort = False)
df_mp['Industry'] = 'TOTAL INDUSTRIES'
df_mp['Database'] = MP[MP_sn[0]].columns[0]
df_mp = df_mp.drop(df_mp[df_mp['Year'] == 'year' ].index, axis = 0)\
             .merge(MP_key, how = 'left', on = 'Key')

In [13]:
df_mp.shape

(375687, 7)

# Penn World Table version 9.1

Database description: PWT version 9.1 is a database with information on relative levels of income, output, input and productivity, covering 182 countries between 1950 and 2017.

The code downloads the Penn World Table version 9.1 database from https://www.rug.nl/ggdc/productivity/pwt/ and convert it into a pandas dataframe.

In [14]:
pw_home = requests.get('https://www.rug.nl/ggdc/productivity/pwt/')
pw_excel_files = [i.get('href') for i in BeautifulSoup(pw_home.content).find_all('a')\
               if i.get('href') is not None and '.xlsx' in i.get('href')]

In [15]:
pw_sn = pd.ExcelFile('https://www.rug.nl' + pw_excel_files[0]).sheet_names
pw = pd.read_excel('https://www.rug.nl' + pw_excel_files[0],sheet_name = pw_sn)

In [16]:
pw_key = pw[pw_sn[1]].dropna(axis = 0, subset = ['Variable definition'])\
                     .rename({'Variable name':'Key','Variable definition':'Variable'},axis = 1)

In [17]:
df_pw = pd.melt(pw[pw_sn[2]].drop('currency_unit', axis = 1).drop('countrycode', axis = 1),\
                id_vars = ['country','year'])\
                .rename({'country':'Country','year':'Year','variable':'Key','value':'Value'},axis = 1)\
                .merge(pw_key, how = 'left', on = 'Key')
df_pw['Industry'] = 'TOTAL INDUSTRIES'
df_pw['Database'] = pw[pw_sn[0]].columns[0]

# Merging the dataframes

In [18]:
df = pd.concat([df_mp,df_pw,df_euklems],sort = False)

In [25]:
df['Variable'].value_counts()

Real GDP per capita in 2011US$, multiple benchmarks (suitable for cross-country income comparisons)            125229
Population, mid-year (thousands)                                                                               125229
Real GDP per capita in 2011US$, 2011 benchmark (suitable for cross-country growth comparisons)                 125229
Gross value added at current basic prices (in millions of national currency)                                    32298
Number of persons engaged (thousands)                                                                           31080
Compensation of employees (in millions of national currency)                                                    30240
Intermediate inputs at current purchasers prices (in millions of national currency)                             30198
Gross Output at current basic prices (in millions of national currency)                                         30198
Number of employees (thousands)                         

In [47]:
df.loc[(df['Variable'] == 'Real gross fixed capital formation volume (2010 prices) of All assets') \
                         & (df['Industry'] == 'TOTAL INDUSTRIES')]

,Country,Key,Year,Industry,Value,Database,Variable
53004,Austria,Iq_GFCF,1995,TOTAL INDUSTRIES,55982.3,EU KLEMS,Real gross fixed capital formation volume (201...
53046,Austria,Iq_GFCF,1996,TOTAL INDUSTRIES,58432.2,EU KLEMS,Real gross fixed capital formation volume (201...
53088,Austria,Iq_GFCF,1997,TOTAL INDUSTRIES,58818.5,EU KLEMS,Real gross fixed capital formation volume (201...
53130,Austria,Iq_GFCF,1998,TOTAL INDUSTRIES,60813.3,EU KLEMS,Real gross fixed capital formation volume (201...
53172,Austria,Iq_GFCF,1999,TOTAL INDUSTRIES,61516.2,EU KLEMS,Real gross fixed capital formation volume (201...
53214,Austria,Iq_GFCF,2000,TOTAL INDUSTRIES,65117.3,EU KLEMS,Real gross fixed capital formation volume (201...
53256,Austria,Iq_GFCF,2001,TOTAL INDUSTRIES,64286.3,EU KLEMS,Real gross fixed capital formation volume (201...
53298,Austria,Iq_GFCF,2002,TOTAL INDUSTRIES,62403.2,EU KLEMS,Real gross fixed capital formation volume (201...
53340,Austria,Iq_GFCF,2003,TOTAL INDUSTRIES,64777.9,EU KLEMS,Real gross fixed capital formation volume (201...
53382,Austria,Iq_GFCF,2004,TOTAL INDUSTRIES,65375.6,EU KLEMS,Real gross fixed capital formation volume (201...


In [16]:
df.groupby(['Country', 'Year'])